Классификация эмоциональной окраски узбекских текстов (0–5)

In [ ]:
label2emotion = {
    0: "Sadness (Грусть)",
    1: "Joy (Радость)",
    2: "Love (Любовь)",
    3: "Anger (Злость)",
    4: "Shame/Fear (Стыд/Страх)",
    5: "Surprise (Удивление)",
}

In [ ]:
pip install  scikit-learn pandas joblib

In [ ]:
!git clone https://github.com/MaksudSharipov/UzbekLemmatizer.git

Cloning into 'UzbekLemmatizer'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 54 (delta 19), reused 37 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 12.17 KiB | 2.43 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
import pandas as pd
import string
import re
import joblib
from UzbekLemmatizer.UzbekLemmatizer import Lemma
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Загрузка данных
train_df = pd.read_csv("data/raw/train.csv")
val_df = pd.read_csv("data/raw/validation.csv")

# Загрузка стоп-слов
def load_stopwords(file_path):
    stopwords = set()
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue
            if '\t' in line:
                parts = line.split('\t')
                word = parts[-1].strip()
            else:
                word = line.split('.', 1)[-1].strip()
            if word:
                stopwords.add(word)
    return stopwords
# https://zenodo.org/records/5659638
uz_stopwords = load_stopwords('data/uzbek_stopwords/stopwords_unigrams.txt')

# Stanza (StanfordNLP)
# spaCy
# Hugging Face Transformers (XLM-RoBERTa, mBERT)

# Предобработка с использованием UzbekLemmatizer
def preprocess(text):
    text = str(text).lower()
    text = re.sub(r"[^\w\sʼʻ’]", " ", text) # удаляем пунктуацию
    text = re.sub(r"[ʼʻ’]", "'", text) # нормализуем апострофы
    tokens = text.split()
    lemmas = []
    for word in tokens:
        result = Lemma(word, full=True)
        if isinstance(result, list) and len(result) >= 2:
            lemma = result[1]
            if lemma and lemma not in uz_stopwords:
                lemmas.append(lemma)
    return " ".join(lemmas) if lemmas else "token"

# def preprocess(text):
#     text = str(text).lower()
#     text = re.sub(r"[^\w\sʼʻ’]", " ", text)   # удаляем пунктуацию
#     text = re.sub(r"[ʼʻ’]", "'", text)        # нормализуем апострофы
#     tokens = text.split()
#     tokens = [word for word in tokens if word and word not in uz_stopwords]
#     return " ".join(tokens) if tokens else "token"



# Применение предобработки
train_df["uzbek_text"] = train_df["uzbek_text"].apply(preprocess)
val_df["uzbek_text"] = val_df["uzbek_text"].apply(preprocess)

print(train_df["uzbek_text"].head(10))


# Pipeline
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("svm", SVC())
])

# Поиск гиперпараметров
param_grid = {
    "svm__C": [0.1, 1, 10],
    "svm__kernel": ["linear", "rbf"],
    "svm__gamma": ["scale", "auto"]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring="accuracy", verbose=1, n_jobs=-1)
grid_search.fit(train_df["uzbek_text"], train_df["label"])

# Лучшие параметры
best_params = grid_search.best_params_

# Финальная модель
final_model = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("svm", SVC(**{
        "C": best_params["svm__C"],
        "kernel": best_params["svm__kernel"],
        "gamma": best_params["svm__gamma"]
    }))
])

final_model.fit(train_df["uzbek_text"], train_df["label"])

# Оценка на validation
y_pred = final_model.predict(val_df["uzbek_text"])
print("\n📊 Classification Report:")
print(classification_report(val_df["label"], y_pred))

# Сохранение модели
joblib.dump(final_model, "final_svm_model.joblib")


0                     o zimni kamsitilgan his qilmadim
1    g amxo rlik qiladigan hushyor lgan odamning li...
2    post yozishga daqiqa vaqt ajratyapman o zimni ...
3    kamin nostaljik his qilaman mulkda ekanligini ...
4                        o zimni g amgin his qilyapman
5    oxirgi paytlarda o zimni og ir his qildim bilm...
6    tavsiya etilgan miqdorda milligramm marta qabu...
7    o zimni o smirlikdagidek chalkash yoshli odamd...
8    petronas ko p yillar davomida ishlayman petron...
9                       o zimni romantik his qilyapman
Name: uzbek_text, dtype: object
Fitting 3 folds for each of 12 candidates, totalling 36 fits

📊 Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.80      0.72       550
           1       0.70      0.80      0.75       704
           2       0.75      0.43      0.55       178
           3       0.74      0.47      0.57       275
           4       0.65      0.54      0.59       212
      

['final_svm_model.joblib']

Выводы:
Лемматизация дала положительный эффект: благодаря предварительной нормализации слов уменьшено влияние морфологических вариаций.

Лучшие результаты достигнуты по классам 0 и 1 (F1 > 0.72), что указывает на более чёткие признаки в этих категориях.

Хуже всего модель справляется с классами 2 и 3, вероятно, из-за их морфологической или семантической схожести с другими классами.

В целом, модель показала средний уровень качества, что даёт возможность улучшения за счёт:

увеличения объёма тренировочных данных,

тонкой настройки параметров.